In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from typing import List, Dict, Union
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from openai import OpenAI
from metrics import *
from agent import *
from prompt import *
from scipy.spatial.distance import jensenshannon
from collections import defaultdict

/home/yl3427/miniconda3/envs/llm_env/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
t_train_df = pd.read_csv(
    "/home/yl3427/cylab/selfCorrectionAgent/result/t14_train_0.csv"
).sort_values(by="patient_filename")
t_zscot_df = pd.read_csv(
    "/home/yl3427/cylab/selfCorrectionAgent/result/0716_t14_zscot_test_800.csv"
).sort_values(by="patient_filename")
print(len(t_train_df))
split_ids = t_train_df.patient_filename
t_zscot_df = t_zscot_df[t_zscot_df.patient_filename.isin(split_ids)]
print(len(t_zscot_df))

100
100


In [3]:
true_dict = {"T1": [], "T2": [], "T3": [], "T4": []}
false_dict = {"T1": [], "T2": [], "T3": [], "T4": []}
# true_dict = defaultdict(list)
# false_dict = defaultdict(list)

for idx, (filename, label, zscot_ans, zscot_rsn) in enumerate(
    zip(
        t_zscot_df.patient_filename,
        t_zscot_df.t,
        t_zscot_df.zs_t_ans_str,
        t_zscot_df.zs_t_reasoning,
    )
):
    zscot_ans = zscot_ans.upper()
    data = {
        "Patient filename": filename,
        "report": t_zscot_df[t_zscot_df.patient_filename == filename].text.values[0],
        "ZS Reasoning": zscot_rsn,
        "Answer": f"T{label+1}",
        "ZS Answer": zscot_ans,
    }
    if f"T{label+1}" in zscot_ans:
        true_dict[f"T{label+1}"].append(data)
        # true_dict[zscot_ans].append(data)
    else:
        false_dict[f"T{label+1}"].append(data)
        # false_dict[zscot_ans].append(data)

In [4]:
len(true_dict["T1"]), len(false_dict["T1"]), len(true_dict["T2"]), len(
    false_dict["T2"]
), len(true_dict["T3"]), len(false_dict["T3"]), len(true_dict["T4"]), len(
    false_dict["T4"]
)

(11, 8, 61, 1, 12, 4, 2, 1)

In [6]:
for key in false_dict.keys():
    print(key)
    for idx, case in enumerate(false_dict[key]):
        print(f"{idx}. {case['ZS Reasoning']}")
        print()
    print("-------------------")

T1
0. The T stage of a cancer is determined by the size of the primary tumor. In this report, the primary tumor in the left breast is described as a 'firm, white, well-circumscribed mass measuring 1.0 x 1.0 x 0.9 cm, located within the upper outer quadrant at the two o'clock aspect, 1.2 cm from the deep margin.' This indicates that the tumor is larger than 1 cm but not larger than 2 cm, which corresponds to a T2 stage. The presence of ductal carcinoma in situ (DCIS) and lobular carcinoma in situ (LCIS) do not affect the T stage. Therefore, the T stage for this patient is T2.

1. The T stage of a cancer is determined by the size of the tumor and whether it has invaded nearby tissue. In this pathology report, the left breast specimen has a centrally placed mass (T1) that measures 1.7 x 1.5 x 1.5 cm and a second mass (T2) that is laterally placed and measures 1.0 cm in diameter with a central cavity filled with necrotic material. The T2 mass is the larger of the two masses, with the great

In [8]:
n_train_df = pd.read_csv(
    "/home/yl3427/cylab/selfCorrectionAgent/result/n03_train_1.csv"
).sort_values(by="patient_filename")
n_zscot_df = pd.read_csv(
    "/home/yl3427/cylab/selfCorrectionAgent/result/0716_n03_zscot_test_800.csv"
).sort_values(by="patient_filename")
print(len(n_train_df))
split_ids = n_train_df.patient_filename
n_zscot_df = n_zscot_df[n_zscot_df.patient_filename.isin(split_ids)]
print(len(n_zscot_df))

100
100


In [9]:
true_dict = {"N0": [], "N1": [], "N2": [], "N3": []}
false_dict = {"N0": [], "N1": [], "N2": [], "N3": []}
# true_dict = defaultdict(list)
# false_dict = defaultdict(list)

for idx, (filename, label, zscot_ans, zscot_rsn) in enumerate(
    zip(
        n_zscot_df.patient_filename,
        n_zscot_df.n,
        n_zscot_df.zs_n_ans_str,
        n_zscot_df.zs_n_reasoning,
    )
):
    zscot_ans = zscot_ans.upper().replace("NO", "N0").replace("NL", "N1")
    data = {
        "Patient filename": filename,
        "report": n_zscot_df[n_zscot_df.patient_filename == filename].text.values[0],
        "ZS Reasoning": zscot_rsn,
        "Answer": f"N{label}",
        "ZS Answer": zscot_ans,
    }
    if f"N{label}" in zscot_ans:
        true_dict[f"N{label}"].append(data)
        # true_dict[zscot_ans].append(data)
    else:
        false_dict[f"N{label}"].append(data)
        # false_dict[zscot_ans].append(data)

In [10]:
len(true_dict["N0"]), len(false_dict["N0"]), len(true_dict["N1"]), len(
    false_dict["N1"]
), len(true_dict["N2"]), len(false_dict["N2"]), len(true_dict["N3"]), len(
    false_dict["N3"]
)

(36, 2, 37, 4, 10, 2, 7, 2)

In [15]:
for key in false_dict.keys():
    print(key)
    for idx, case in enumerate(false_dict[key]):
        print(f"{idx}. {case['ZS Reasoning']}")
        print(case["Answer"])
        print()
    print("-------------------")

N0
0. The report mentions 'Lymphonodultis chronica et sinus histiocytosis (No. VI)' in the axillary lymph nodes. This indicates that 6 lymph nodes were found to contain metastatic cancer. The N stage for breast cancer is determined by the number of involved lymph nodes, with N0 indicating no cancer in the lymph nodes and N3 indicating extensive spread to the lymph nodes. In this case, since 6 lymph nodes were involved, the N stage would be N1.
N0

1. The report mentions '3.7 cm to the axilla' which indicates the presence of cancer in 1 to 3 axillary lymph nodes, corresponding to N1 stage. The substaging information is not provided, so we can only determine the main N stage.
N0

-------------------
N1
0. The N stage for cancer is determined by the number of lymph nodes involved. In this report, metastatic adenocarcinoma is identified in 3 of 27 right axillary lymph nodes. Therefore, the N stage is N2.
N1

1. The report states that multiple (14) lymph nodes were removed from the left axi

In [4]:
t_KEwLTM_file_path = "/home/yl3427/cylab/selfCorrectionAgent/result/0720_t14_dynamic_test_0_outof_10runs_verification.csv"
t_KEwLTM_df = pd.read_csv(t_KEwLTM_file_path)

t_zscot_file_path = "/home/yl3427/cylab/selfCorrectionAgent/result/0720_t14_zscot_test_800_0_verification.csv"
t_zscot_df = pd.read_csv(t_zscot_file_path)
t_zscot_df = t_zscot_df[t_zscot_df.patient_filename.isin(t_KEwLTM_df.patient_filename)]

In [5]:
t_KEwLTM_df.columns, t_zscot_df.columns

(Index(['patient_filename', 't', 'text', 'cmem_t_10reports_is_parsed',
        'cmem_t_10reasoning', 'cmem_t_10reports_ans_str',
        'cmem_t_20reports_is_parsed', 'cmem_t_20reasoning',
        'cmem_t_20reports_ans_str', 'cmem_t_30reports_is_parsed',
        'cmem_t_30reasoning', 'cmem_t_30reports_ans_str',
        'cmem_t_40reports_is_parsed', 'cmem_t_40reasoning',
        'cmem_t_40reports_ans_str', 'cmem_t_50reports_is_parsed',
        'cmem_t_50reasoning', 'cmem_t_50reports_ans_str',
        'cmem_t_60reports_is_parsed', 'cmem_t_60reasoning',
        'cmem_t_60reports_ans_str', 'cmem_t_70reports_is_parsed',
        'cmem_t_70reasoning', 'cmem_t_70reports_ans_str',
        'cmem_t_80reports_is_parsed', 'cmem_t_80reasoning',
        'cmem_t_80reports_ans_str', 'cmem_t_90reports_is_parsed',
        'cmem_t_90reasoning', 'cmem_t_90reports_ans_str',
        'cmem_t_100reports_is_parsed', 'cmem_t_100reasoning',
        'cmem_t_100reports_ans_str', 't_evaluation'],
       dtype='objec

In [3]:
t_groundtruth_issue = {
    "TCGA-B6-A0IE.DFCA9C6E-710E-4645-9CFC-A908AAD583F3",
    "TCGA-JL-A3YX.25782EF0-8786-446E-ADBA-21F489844237",
    "TCGA-B6-A0IE.DFCA9C6E-710E-4645-9CFC-A908AAD583F3",
    "TCGA-BH-A208.4F943D12-E769-45F3-86BE-75193786DD4E",
    "TCGA-AO-A0J9.1E3F3136-6D86-4470-85AA-55B11C9E24CD",
    "TCGA-BH-A1FM.DA6A0EC9-6E20-4E4A-9B7F-A32EFF7627AD",
    "TCGA-D8-A1JS.0EA57ABF-E3DA-4862-BAB8-A6E36408AC42",
    "TCGA-JL-A3YX.25782EF0-8786-446E-ADBA-21F489844237",
    "TCGA-BH-A1FM.DA6A0EC9-6E20-4E4A-9B7F-A32EFF7627AD",
}
t_groundtruth_issue

{'TCGA-AO-A0J9.1E3F3136-6D86-4470-85AA-55B11C9E24CD',
 'TCGA-B6-A0IE.DFCA9C6E-710E-4645-9CFC-A908AAD583F3',
 'TCGA-BH-A1FM.DA6A0EC9-6E20-4E4A-9B7F-A32EFF7627AD',
 'TCGA-BH-A208.4F943D12-E769-45F3-86BE-75193786DD4E',
 'TCGA-D8-A1JS.0EA57ABF-E3DA-4862-BAB8-A6E36408AC42',
 'TCGA-JL-A3YX.25782EF0-8786-446E-ADBA-21F489844237'}

In [15]:
t_test_df = pd.read_csv(
    "/home/yl3427/cylab/selfCorrectionAgent/result/0720_t14_dynamic_test_0_outof_10runs_verification.csv"
).sort_values(by="patient_filename")
t_zscot_df = pd.read_csv(
    "/home/yl3427/cylab/selfCorrectionAgent/result/0720_t14_zscot_test_800_0_verification.csv"
).sort_values(by="patient_filename")
split_ids = t_test_df.patient_filename
t_zscot_df = t_zscot_df[t_zscot_df.patient_filename.isin(split_ids)]

# output_dir = "/secure/shared_data/studio_label/zscot_error_t_verified"
# output_dir = "/secure/shared_data/studio_label/kewltm_error_t_verified"
output_dir = "/secure/shared_data/studio_label/both_error_t_verified"
os.makedirs(output_dir, exist_ok=True)

ids_set = set()
for idx, (
    filename,
    label,
    memory_ans,
    zscot_ans,
    memory_rsn,
    zscot_rsn,
    memory_ev,
    zscot_ev,
) in enumerate(
    zip(
        t_test_df.patient_filename,
        t_test_df.t,
        t_test_df.cmem_t_40reports_ans_str,
        t_zscot_df.zs_t_ans_str,
        t_test_df.cmem_t_40reasoning,
        t_zscot_df.zs_t_reasoning,
        t_test_df.t_evaluation,
        t_zscot_df.t_evaluation,
    )
):
    memory_ans = memory_ans.upper()
    zscot_ans = zscot_ans.upper()
    if filename in t_groundtruth_issue:
        continue
    # if (f"T{label+1}" not in zscot_ans) and (f"T{label+1}" in memory_ans): # cases where only memory was correct
    # if (f"T{label+1}" in zscot_ans) and (f"T{label+1}" not in memory_ans): # cases where only zs was correct
    if (f"T{label+1}" not in zscot_ans) and (
        f"T{label+1}" not in memory_ans
    ):  # cases where both were wrong
        ids_set.add(filename)
        data = {
            "data": {
                "humanMachineDialogue": [
                    {"author": "Patient filename", "text": filename},
                    {"author": "Memory Reasoning", "text": memory_rsn},
                    {"author": "ZS Reasoning", "text": zscot_rsn},
                    {"author": "Answer", "text": f"T{label+1}"},
                    {"author": "Memory Answer", "text": memory_ans},
                    {"author": "ZS Answer", "text": zscot_ans},
                    {"author": "Memory Evaluation", "text": memory_ev},
                    {"author": "ZS Evaluation", "text": zscot_ev},
                ]
            }
        }

        print(json.dumps(data, indent=4))
        file_name = f"t_{idx}.json"
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, "w") as json_file:
            json.dump(data, json_file, indent=4)

{
    "data": {
        "humanMachineDialogue": [
            {
                "author": "Patient filename",
                "text": "TCGA-5L-AAT0.F9B6971F-23C0-465F-BFEC-778BF228A1AE"
            },
            {
                "author": "Memory Reasoning",
                "text": "The report indicates that the tumor is invasive and measures 2.2 cm in the largest diameter, which falls within the criteria for T1 (invasive tumor size 2 cm or less). The fact that the tumor is invasive lobular carcinoma and its size cannot be accurately determined does not apply here, as the size is accurately measured and is less than 2 cm. Therefore, the T stage for this patient's cancer is T1."
            },
            {
                "author": "ZS Reasoning",
                "text": "The report indicates that the largest dimension of the tumor is 0.4 cm, which falls within the range for T1 stage cancers. T1 cancers are generally 2 cm or less in size. Therefore, based on the information provided 

In [16]:
len(ids_set)

58

In [17]:
n_groundtruth_issue = {
    "TCGA-BH-A1FJ.8169BE67-03C8-4F4D-9A60-200705B795AE",
    "TCGA-BH-A6R9.1DB8FAFB-FC4A-4401-8316-30FB5352335D",
    "TCGA-E9-A1QZ.864BB34A-1008-480C-A3B5-A2C616E95C49",
    "TCGA-A8-A06Z.956F45E5-A8C6-4A4A-9D1F-D31912180584",
    "TCGA-B6-A0IK.3A38A97C-2CBB-4802-9528-A4BBD62AEA4A",
    "TCGA-B6-A0WV.506BFD3B-240B-440E-B7A0-E596FC0B7F72",
    "TCGA-E9-A3X8.00058FFD-35E6-4891-8B01-DAB3AE9EBF78",
    "TCGA-GM-A2DA.F3CD8E6E-B02F-4D5D-B895-6DF063F61603",
    "TCGA-B6-A0IH.12C64846-1CB3-42E4-B307-54C7AD12F530",
    "TCGA-B6-A0WW.F05F5886-DC5D-4685-B2BF-57A68A0BB7B9",
    "TCGA-B6-A0X1.D792031E-2CCE-4341-B3B3-C7D1D84F8F6B",
    "TCGA-GM-A2DA.F3CD8E6E-B02F-4D5D-B895-6DF063F61603",
    "TCGA-B6-A1KN.72996825-1FFA-4C51-8DB0-DA74BCB595EB",
}
n_groundtruth_issue

{'TCGA-A8-A06Z.956F45E5-A8C6-4A4A-9D1F-D31912180584',
 'TCGA-B6-A0IH.12C64846-1CB3-42E4-B307-54C7AD12F530',
 'TCGA-B6-A0IK.3A38A97C-2CBB-4802-9528-A4BBD62AEA4A',
 'TCGA-B6-A0WV.506BFD3B-240B-440E-B7A0-E596FC0B7F72',
 'TCGA-B6-A0WW.F05F5886-DC5D-4685-B2BF-57A68A0BB7B9',
 'TCGA-B6-A0X1.D792031E-2CCE-4341-B3B3-C7D1D84F8F6B',
 'TCGA-B6-A1KN.72996825-1FFA-4C51-8DB0-DA74BCB595EB',
 'TCGA-BH-A1FJ.8169BE67-03C8-4F4D-9A60-200705B795AE',
 'TCGA-BH-A6R9.1DB8FAFB-FC4A-4401-8316-30FB5352335D',
 'TCGA-E9-A1QZ.864BB34A-1008-480C-A3B5-A2C616E95C49',
 'TCGA-E9-A3X8.00058FFD-35E6-4891-8B01-DAB3AE9EBF78',
 'TCGA-GM-A2DA.F3CD8E6E-B02F-4D5D-B895-6DF063F61603'}

In [26]:
n_test_df = pd.read_csv(
    "/home/yl3427/cylab/selfCorrectionAgent/result/0720_n03_dynamic_test_1_outof_10runs_verification.csv"
).sort_values(by="patient_filename")
n_zscot_df = pd.read_csv(
    "/home/yl3427/cylab/selfCorrectionAgent/result/0720_n03_zscot_test_800_1_verification.csv"
).sort_values(by="patient_filename")
split_ids = n_test_df.patient_filename
n_zscot_df = n_zscot_df[n_zscot_df.patient_filename.isin(split_ids)]

# output_dir = "/secure/shared_data/studio_label/zscot_error_n_verified"
# output_dir = "/secure/shared_data/studio_label/kewltm_error_n_verified"
output_dir = "/secure/shared_data/studio_label/both_error_n_verified"
os.makedirs(output_dir, exist_ok=True)

ids_set = set()
for idx, (
    filename,
    label,
    memory_ans,
    zscot_ans,
    memory_rsn,
    zscot_rsn,
    memory_ev,
    zscot_ev,
) in enumerate(
    zip(
        n_test_df.patient_filename,
        n_test_df.n,
        n_test_df.cmem_n_40reports_ans_str,
        n_zscot_df.zs_n_ans_str,
        n_test_df.cmem_n_40reasoning,
        n_zscot_df.zs_n_reasoning,
        n_test_df.n_evaluation,
        n_zscot_df.n_evaluation,
    )
):
    if filename in n_groundtruth_issue:
        continue

    memory_ans = memory_ans.upper().replace("NO", "N0").replace("NL", "N1")
    zscot_ans = zscot_ans.upper().replace("NO", "N0").replace("NL", "N1")

    # if (f"N{label}" not in zscot_ans) and (f"N{label}" in memory_ans): # cases where only memory was correct
    # if (f"N{label}" in zscot_ans) and (f"N{label}" not in memory_ans): # cases where only zs was correct
    if (f"N{label}" not in zscot_ans) and (
        f"N{label}" not in memory_ans
    ):  # cases where both were wrong
        ids_set.add(filename)
        data = {
            "data": {
                "humanMachineDialogue": [
                    {"author": "Patient filename", "text": filename},
                    {"author": "Memory Reasoning", "text": memory_rsn},
                    {"author": "ZS Reasoning", "text": zscot_rsn},
                    {"author": "Answer", "text": f"N{label}"},
                    {"author": "Memory Answer", "text": memory_ans},
                    {"author": "ZS Answer", "text": zscot_ans},
                    {"author": "Memory Evaluation", "text": memory_ev},
                    {"author": "ZS Evaluation", "text": zscot_ev},
                ]
            }
        }

        print(json.dumps(data, indent=4))
        file_name = f"n_{idx}.json"
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, "w") as json_file:
            json.dump(data, json_file, indent=4)

{
    "data": {
        "humanMachineDialogue": [
            {
                "author": "Patient filename",
                "text": "TCGA-A2-A0CK.B065FC65-CD33-4878-AE2C-7E8C04F5ECAB"
            },
            {
                "author": "Memory Reasoning",
                "text": "The pathology report does not mention any regional lymph node metastasis. The report only mentions that the surgical margins are positive for tumor and provides details about the tumor size, type, and other factors. Therefore, based on the given rules, the N stage for this patient is N0."
            },
            {
                "author": "ZS Reasoning",
                "text": "The pathology report does not mention any lymph node involvement in the pathologic N staging. The report specifically states 'AJCC Staging: STAGE I (T1, NX, MX),' where 'NX' indicates that the lymph nodes cannot be assessed. In this case, the 'N' stage is 'N0', which means no regional lymph node metastasis."
            },
   

In [27]:
len(ids_set)

61